# Merge Root Files or Create CSV

## Merge Root Files

In [1]:
import uproot
import os
import numpy as np

def merge_files(directory, starts_with, output_name):

    file_list = []

    for file in os.listdir(directory):
        if file.endswith('.root') and not file.startswith('merge') and not file.startswith(output_name):
            if not starts_with == '' and file.startswith(starts_with):
                    file_list.append(os.path.join(directory, file))
            else:
                file_list.append(os.path.join(directory, file))
    
    merged_file = os.path.join(directory, output_name)
    
    counter = 0
    while True:
        if not os.path.exists(f"{merged_file}{counter}.root"):
            merged_file = f"{merged_file}{counter}.root"
            break
        counter = counter + 1

    with uproot.recreate(merged_file) as f_out:
        data_dict = {}  # Dictionary to store merged data temporarily
        
        for file in file_list:
            
            with uproot.open(file) as f_in:
                for key in f_in.keys():
                    obj = f_in[key]
                    
                    if isinstance(obj, uproot.TTree):
                        new_data = obj.arrays(library="np")
                        
                        # Extract base key name (ignore cycle numbers)
                        base_key = key.split(';')[0]

                        # If base_key is already in data_dict, concatenate data
                        if base_key in data_dict:
                            existing_data = data_dict[base_key]
                            combined_data = {k: np.concatenate([existing_data[k], new_data[k]]) for k in new_data.keys() if k in existing_data}
                            # Update with the combined data
                            data_dict[base_key] = {**existing_data, **combined_data}
                        else:
                            # If base_key is not in data_dict, add new data
                            data_dict[base_key] = new_data

        for key, data in data_dict.items():
            f_out[key] = data


    print("Merged files into", merged_file)

In [3]:
directory = 'BUILD/ROOT'
starts_with = 'root'
output_name = 'merge'

merge_files(directory, starts_with, output_name)

Merged files into BUILD/ROOT/merge0.root


## Draw Graph from .root File

In [1]:
import uproot
import matplotlib.pyplot as plt
import os

def scatterplot_from_root(directory, input_file, tree_name, x_branch, y_branch, title, x_label, y_label, output_image):

    input_file_path = os.path.join(directory, input_file)
    
    with uproot.open(input_file_path) as file:
        tree = file[tree_name]
        
        x_values = tree[x_branch].array()
        y_values = tree[y_branch].array()

    plt.figure(figsize=(8, 6))
    plt.scatter(x_values, y_values, color='blue', s=10, edgecolor='black')
    
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)

    plt.savefig(output_image + '.jpg', format='jpg', dpi = 300)
    plt.close()
    
    print(f' ===== Scatter plot saved as {output_image}.jpg =====')

In [ ]:
directory = 'build/root'
file_name = 'merged0' + '.root'

tree_name = "Photons"
x_branch = "X_axis"
y_branch = "Y_axis"

title = "title"
x_label = "x axis"
y_label = "y axis"

output_image = "xy_plot3"

scatterplot_from_root(directory, file_name, tree_name, x_branch, y_branch, title, x_label, y_label, output_image)

## Extract .root -> CSV File

In [10]:
import uproot
import os
import csv
import numpy as np

def copy_data_csv(directory, root_name_starts, tree_name, 
                  x_branch, y_branch, z_branch, decimal_places, csv_save_as):
    csv_file = os.path.join(directory, csv_save_as)
    
    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([x_branch, y_branch])
        # writer.writerow(['File', tree_name, x_branch, y_branch, z_branch])
        
        count = 0
        for file_name in sorted(os.listdir(directory)):

            if file_name.endswith('.root') and file_name.startswith(root_name_starts):
                file_path = os.path.join(directory, file_name)

                try:
                    # Open the ROOT file using uproot
                    with uproot.open(file_path) as root_file:
                        tree = root_file[tree_name]
                        if tree is None and count == 0:
                            print(f"Tree '{tree_name}' not found in {file_path}")
                            count += 1
                            continue

                        # Extract branches as arrays
                        x_values = tree[x_branch].array() if x_branch in tree else None
                        y_values = tree[y_branch].array() if y_branch in tree else None
                        z_values = tree[z_branch].array() if z_branch in tree else None

                        x_values = np.round(x_values, decimal_places) if x_branch in tree else None
                        y_values = np.round(y_values, decimal_places) if x_branch in tree else None

                        # Write data to CSV
                        for i in range(len(x_values)):
                            row = [x_values[i]]
                            # row = [file_name, tree_name, x_values[i]]
                            if y_values is not None:
                                row.append(y_values[i])
                            if z_values is not None:
                                row.append(z_values[i])
                            writer.writerow(row)

                except Exception as e: print(f"Error processing {file_path}: {e}")
                
    print('CSV file created as:', csv_file)

In [ ]:
directory = 'build/root/'
root_name_starts = "merge2"

tree_name = "Photons"
x_branch  = "X_axis"
y_branch  = "Y_axis"
z_branch  = ""

decimal_places = 2

csv_save_as = 'problema' + '.csv'

copy_data_csv(directory, root_name_starts, tree_name, 
              x_branch, y_branch, z_branch, decimal_places, csv_save_as)